## Этот код выполняет классификацию электронных писем на спам и не спам с использованием метода Multinomial Naive Bayes. Он используется для оценки производительности классификатора на тестовом наборе данных, вычисляя такие метрики, как точность (precision), полнота (recall), F1-мера (F1-score) и общая точность классификации (classification accuracy). В результате вы получаете оценку эффективности классификации модели на задаче определения спама в электронной почте.

In [1]:
import os  # Модуль для работы с операционной системой
import email_read_util  # Модуль для чтения и обработки электронных писем
from sklearn.model_selection import train_test_split  # Модуль для разделения набора данных на обучающий и тестовый
from sklearn.feature_extraction.text import CountVectorizer  # Модуль для извлечения признаков из текста
from sklearn.naive_bayes import MultinomialNB  # Модуль для реализации наивного байесовского классификатора
from sklearn.metrics import accuracy_score, classification_report  # Модуль для оценки качества классификации

## Download 2007 TREC Public Spam Corpus
1. Read the "Agreement for use"
   https://plg.uwaterloo.ca/~gvcormac/treccorpus07/

2. Download 255 MB Corpus (trec07p.tgz) and untar into the 'chapter1/datasets' directory

3. Check that the below paths for 'DATA_DIR' and 'LABELS_FILE' exist

## Определение директории данных и файлов меток:
### В этом блоке определяются пути к директории с данными и файлу с метками. Переменные DATA_DIR и LABELS_FILE содержат пути к директории и файлу, соответственно. Также задается параметр TRAINING_SET_RATIO, который определяет пропорцию разделения данных на обучающий и тестовый наборы.

In [2]:
# Определение директории с данными и файла с метками
DATA_DIR = 'datasets/trec07p/data/'  # Путь к директории с данными
LABELS_FILE = 'datasets/trec07p/full/index'  # Путь к файлу с метками
TRAINING_SET_RATIO = 0.7  # Пропорция разделения набора данных на тренировочный и тестовый
labels = {}  # Словарь для хранения меток (имя файла: метка)

## Чтение и обработка меток:
### Далее происходит чтение и обработка файла с метками. С помощью цикла for считываются метки из файла LABELS_FILE и сохраняются в словаре labels, где ключами являются имена файлов, а значениями - метки

In [3]:
# Чтение и обработка файла меток
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

## Функция для чтения электронных писем:
### Здесь определяется функция read_email_files(), которая читает и обрабатывает электронные письма. В цикле for происходит итерация по количеству меток, и для каждой метки считывается соответствующее письмо из директории DATA_DIR с помощью функции extract_email_text из модуля email_read_util. Текст письма добавляется в список X, а его метка - в список y.

In [4]:
def read_email_files():
    X = []
    y = [] 
    for i in range(len(labels)):
        filename = 'inmail.' + str(i+1)
        email_str = email_read_util.extract_email_text(
            os.path.join(DATA_DIR, filename))
        X.append(email_str)
        y.append(labels[filename])
    return X, y

In [5]:
# Чтение и обработка электронных писем
X, y = read_email_files()

## Чтение и обработка данных:
### В этой части кода вызывается функция read_email_files(), результат ее выполнения сохраняется в переменные X и y. После этого данные разделяются на обучающий и тестовый наборы с помощью функции train_test_split, где X и y - это тексты писем и их метки соответственно.

In [6]:
# Разделение набора данных на обучающий и тестовый
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test, idx_train, idx_test = \
    train_test_split(X, y, range(len(y)), 
    train_size=TRAINING_SET_RATIO, random_state=2)

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## Преобразование текста в признаки:
### Для преобразования текста писем в признаки используется класс CountVectorizer. Данные обучающего и тестового наборов преобразуются в матрицы признаков с помощью методов fit_transform и transform соответственно.

In [7]:
# Преобразование текста писем в векторы признаков
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vector = vectorizer.fit_transform(X_train)
X_test_vector = vectorizer.transform(X_test)

## Обучение классификатора и оценка его производительности:
### Далее создается объект классификатора MultinomialNB, который обучается на обучающем наборе с помощью метода fit. Затем выполняется предсказание меток для тестового набора с помощью метода predict. После этого выводится отчет о классификации с помощью функции classification_report и выводится точность классификации с помощью функции accuracy_score.

In [8]:
# Обучение наивного байесовского классификатора
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Инициализируйте классификатор и делайте предсказания меток
mnb = MultinomialNB()
mnb.fit(X_train_vector, y_train)
y_pred = mnb.predict(X_test_vector)

# Вывод результатов классификации
print(classification_report(y_test, y_pred, target_names=['Spam', 'Ham']))
print('Classification accuracy {:.1%}'.format(accuracy_score(y_test, y_pred)))

             precision    recall  f1-score   support

       Spam       0.99      0.94      0.97     15035
        Ham       0.90      0.98      0.94      7591

avg / total       0.96      0.96      0.96     22626

Classification accuracy 95.6%


## Классификационная точность (Classification accuracy) составляет 95.6%, что является общей точностью классификации модели на тестовом наборе данных. Это процент правильно классифицированных объектов относительно общего числа объектов в тестовом наборе.